# So sánh mô hình Embedding: BGE-M3 vs ViEmbedding-base

Notebook này so sánh hiệu suất của hai mô hình:
- BAAI/bge-m3
- anti-ai/ViEmbedding-base

Trên dataset: anti-ai/ViNLI-Zalo-supervised
Sử dụng: TripletLoss và TripletEvaluator

## 1. Cài đặt thư viện

In [ ]:
!pip install sentence-transformers datasets transformers torch accelerate huggingface_hub tqdm pandas numpy scikit-learn

## 2. Import thư viện

In [ ]:
import torch
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
from datetime import datetime
import os

# Kiểm tra GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 3. Tải dataset

In [ ]:
# Tải dataset từ HuggingFace
print("Loading dataset...")
dataset = load_dataset("anti-ai/ViNLI-Zalo-supervised")

print(f"Dataset structure: {dataset}")
print(f"\nSample data:")
print(dataset['train'][0])

# Kiểm tra số lượng samples
if 'train' in dataset:
    print(f"\nTrain samples: {len(dataset['train'])}")
if 'validation' in dataset:
    print(f"Validation samples: {len(dataset['validation'])}")
if 'test' in dataset:
    print(f"Test samples: {len(dataset['test'])}")

## 4. Chuẩn bị dữ liệu cho Triplet Loss

In [ ]:
def prepare_triplet_data(dataset_split):
    """
    Chuyển đổi dataset thành format InputExample cho TripletLoss
    Format: (anchor, positive, negative)
    """
    triplet_examples = []
    
    for sample in tqdm(dataset_split, desc="Preparing triplet data"):
        # Lấy query (anchor), positive, và hard_negative
        query = sample['query']
        positive = sample['positive']
        negative = sample['hard_neg']
        
        # Tạo InputExample với texts chứa [anchor, positive, negative]
        example = InputExample(texts=[query, positive, negative])
        triplet_examples.append(example)
    
    return triplet_examples

# Chuẩn bị dữ liệu
print("\nPreparing training data...")
train_examples = prepare_triplet_data(dataset['train'])

# Nếu có validation set, chuẩn bị cho evaluation
if 'validation' in dataset:
    print("Preparing validation data...")
    val_examples = prepare_triplet_data(dataset['validation'])
elif 'test' in dataset:
    print("Preparing test data...")
    val_examples = prepare_triplet_data(dataset['test'])
else:
    # Nếu không có validation/test, chia train thành 90-10
    print("Splitting train data into train/val (90-10)...")
    split_idx = int(len(train_examples) * 0.9)
    val_examples = train_examples[split_idx:]
    train_examples = train_examples[:split_idx]

print(f"\nTrain examples: {len(train_examples)}")
print(f"Validation examples: {len(val_examples)}")

## 5. Tạo TripletEvaluator

In [ ]:
def create_triplet_evaluator(examples, name="validation"):
    """
    Tạo TripletEvaluator từ list InputExample
    """
    anchors = []
    positives = []
    negatives = []
    
    for example in examples:
        anchors.append(example.texts[0])
        positives.append(example.texts[1])
        negatives.append(example.texts[2])
    
    evaluator = evaluation.TripletEvaluator(
        anchors=anchors,
        positives=positives,
        negatives=negatives,
        name=name
    )
    
    return evaluator

# Tạo evaluator
print("Creating evaluator...")
evaluator = create_triplet_evaluator(val_examples, name="vinli-zalo-eval")
print("Evaluator created successfully!")

## 6. Hàm đánh giá mô hình

In [ ]:
def evaluate_model(model_name, evaluator, device='cuda'):
    """
    Đánh giá một mô hình trên evaluator
    """
    print(f"\n{'='*80}")
    print(f"Evaluating model: {model_name}")
    print(f"{'='*80}")
    
    try:
        # Load model
        print(f"Loading model from {model_name}...")
        model = SentenceTransformer(model_name, device=device)
        
        # Evaluate
        print("Running evaluation...")
        score = evaluator(model)
        
        # Xử lý trường hợp score là dictionary hoặc số
        if isinstance(score, dict):
            accuracy = score.get('accuracy', score.get('cosine_accuracy', 0))
            print(f"\nResults for {model_name}:")
            print(f"Full results: {score}")
            print(f"Accuracy: {accuracy:.4f}")
        else:
            accuracy = score
            print(f"\nResults for {model_name}:")
            print(f"Accuracy: {accuracy:.4f}")
        
        return {
            'model_name': model_name,
            'accuracy': accuracy,
            'full_results': score if isinstance(score, dict) else {'accuracy': score},
            'status': 'success'
        }
        
    except Exception as e:
        print(f"Error evaluating {model_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        return {
            'model_name': model_name,
            'accuracy': None,
            'status': 'failed',
            'error': str(e)
        }

## 7. Đánh giá cả hai mô hình

In [ ]:
# Danh sách mô hình cần đánh giá
models_to_evaluate = [
    "BAAI/bge-m3",
    "anti-ai/ViEmbedding-base"
]

# Đánh giá từng mô hình
results = []

for model_name in models_to_evaluate:
    result = evaluate_model(model_name, evaluator, device=device)
    results.append(result)
    
    # Giải phóng bộ nhớ
    if device == "cuda":
        torch.cuda.empty_cache()

print("\n" + "="*80)
print("Evaluation completed!")
print("="*80)

## 8. Hiển thị kết quả so sánh

In [ ]:
# Tạo DataFrame để hiển thị kết quả
df_results = pd.DataFrame(results)

print("\n" + "="*80)
print("COMPARISON RESULTS")
print("="*80)
print(df_results.to_string(index=False))

# Tìm mô hình tốt nhất
successful_results = [r for r in results if r['status'] == 'success']
if successful_results:
    best_model = max(successful_results, key=lambda x: x['accuracy'])
    print(f"\n🏆 Best Model: {best_model['model_name']}")
    print(f"   Accuracy: {best_model['accuracy']:.4f}")
    
    # Tính độ chênh lệch
    if len(successful_results) == 2:
        diff = abs(successful_results[0]['accuracy'] - successful_results[1]['accuracy'])
        print(f"\n📊 Accuracy difference: {diff:.4f} ({diff*100:.2f}%)")

## 9. Inference chi tiết trên một số mẫu

In [ ]:
def detailed_inference(model_name, examples, num_samples=5, device='cuda'):
    """
    Thực hiện inference chi tiết trên một số mẫu
    """
    print(f"\n{'='*80}")
    print(f"Detailed Inference - Model: {model_name}")
    print(f"{'='*80}")
    
    # Load model
    model = SentenceTransformer(model_name, device=device)
    
    # Chọn ngẫu nhiên một số samples
    sample_indices = np.random.choice(len(examples), min(num_samples, len(examples)), replace=False)
    
    results = []
    
    for idx in sample_indices:
        example = examples[idx]
        query = example.texts[0]
        positive = example.texts[1]
        negative = example.texts[2]
        
        # Encode
        query_emb = model.encode(query, convert_to_tensor=True)
        pos_emb = model.encode(positive, convert_to_tensor=True)
        neg_emb = model.encode(negative, convert_to_tensor=True)
        
        # Tính similarity
        from sentence_transformers import util
        pos_sim = util.cos_sim(query_emb, pos_emb).item()
        neg_sim = util.cos_sim(query_emb, neg_emb).item()
        
        # Xác định đúng/sai
        correct = pos_sim > neg_sim
        
        result = {
            'query': query[:100] + '...' if len(query) > 100 else query,
            'positive': positive[:100] + '...' if len(positive) > 100 else positive,
            'negative': negative[:100] + '...' if len(negative) > 100 else negative,
            'pos_similarity': pos_sim,
            'neg_similarity': neg_sim,
            'correct': correct
        }
        results.append(result)
        
        # In kết quả
        print(f"\nSample {idx + 1}:")
        print(f"Query: {result['query']}")
        print(f"Positive: {result['positive']}")
        print(f"Negative: {result['negative']}")
        print(f"Positive Similarity: {pos_sim:.4f}")
        print(f"Negative Similarity: {neg_sim:.4f}")
        print(f"Result: {'✓ CORRECT' if correct else '✗ INCORRECT'}")
        print("-" * 80)
    
    return results

# Thực hiện inference chi tiết cho cả hai mô hình
print("\n" + "="*80)
print("DETAILED INFERENCE ON SAMPLE DATA")
print("="*80)

inference_results = {}
for model_name in models_to_evaluate:
    try:
        results = detailed_inference(model_name, val_examples, num_samples=5, device=device)
        inference_results[model_name] = results
        
        # Giải phóng bộ nhớ
        if device == "cuda":
            torch.cuda.empty_cache()
    except Exception as e:
        print(f"Error in detailed inference for {model_name}: {str(e)}")

## 10. Lưu kết quả

In [ ]:
# Tạo thư mục kết quả nếu chưa có
output_dir = "./comparison_results"
os.makedirs(output_dir, exist_ok=True)

# Lưu kết quả tổng quát
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = f"{output_dir}/comparison_results_{timestamp}.json"

summary = {
    'timestamp': timestamp,
    'dataset': 'anti-ai/ViNLI-Zalo-supervised',
    'evaluation_metric': 'TripletEvaluator',
    'num_train_samples': len(train_examples),
    'num_val_samples': len(val_examples),
    'models_evaluated': models_to_evaluate,
    'results': results
}

with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

print(f"\nResults saved to: {results_file}")

# Lưu DataFrame
csv_file = f"{output_dir}/comparison_results_{timestamp}.csv"
df_results.to_csv(csv_file, index=False)
print(f"CSV results saved to: {csv_file}")

## 11. Kết luận

In [ ]:
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"\nDataset: anti-ai/ViNLI-Zalo-supervised")
print(f"Training samples: {len(train_examples)}")
print(f"Validation samples: {len(val_examples)}")
print(f"\nModels compared:")
for i, model in enumerate(models_to_evaluate, 1):
    print(f"{i}. {model}")

print(f"\nEvaluation metric: TripletEvaluator (Triplet Accuracy)")
print(f"\nResults have been saved to: {output_dir}")
print("\n" + "="*80)